In [216]:
import json, requests, time
from sqlalchemy import create_engine
import pandas as pd

engine = create_engine('postgresql://postgres:argmax@pg:5432/postgres')

# Postgres vector search
We have seen that postgres is capable of running a k-nearest neighbor search with the cosine similarity metric

## Example
These are the 10 closest apps to the zero vector

In [199]:
vec = [0 for _ in range(1024)]
sql_query = f'''
SELECT
    "bundleId"
FROM app_vectors
ORDER BY embedding<=>'{json.dumps(vec)}'
LIMIT 10
'''
with engine.connect() as db_con:
    df = pd.read_sql(sql_query, con=db_con)

df

,bundleId
0,com.YayySAL.DodgeAgent
1,com.tintash.nailsalon
2,com.tilegarden.match3
3,com.AppIdeas.LevelUpRunner
4,1582745578
5,1569586264
6,com.kamilbilge.ropesavior3d
7,se.ace.fishinc
8,com.volt.dresstoimpress
9,com.loop.match3d


# Embedding new data
We can embed new data with this API call

In [200]:
query_text = "A farming game"
query_vec = requests.get("http://emb:8080/embed", params ={"text": query_text}).json()
print(query_vec)

[0.354161, 0.848121, -0.030284, -0.266633, -0.291369, -0.235802, -0.328547, 0.556077, 0.266265, 0.80911, 0.088049, 0.425911, 0.366686, 0.266709, -0.140682, -0.460998, -0.554848, -0.686548, -0.32312, 0.068696, 0.559805, 0.457103, -0.860526, -0.005353, -0.523179, 0.301315, -0.240179, -0.324041, 0.659639, 0.821582, -0.161332, 0.125691, -0.01421, -0.347641, -0.507953, -0.617259, 0.252902, -0.143336, -0.472049, -0.525402, 0.562244, -0.354457, 0.047799, -0.168605, 0.183767, -0.484756, 0.497892, -0.919021, 0.49797, -0.050206, 0.134188, -0.502712, -0.414337, -0.215174, -0.64883, -0.228308, -0.019863, 0.199011, -0.309924, 0.597036, 1.353649, -0.230922, 0.069312, -0.212216, 0.794984, 0.892149, -0.509315, -0.528928, -0.061134, -0.452766, -0.853116, 0.23235, -0.493771, -0.62472, 0.480888, 0.490589, 0.388387, -0.040939, -0.14579, 0.589986, 0.033822, 0.541735, 0.020558, -0.312139, -0.767122, 0.004014, 0.754478, 0.060997, -0.146129, -0.595552, -0.218311, 0.355091, -0.330841, 0.541868, 0.405484, 0.875

#  Embed the text data within Postgres?
You are were asked to extend Postgres, such that the following syntax would work
## Desired outcome

    SELECT
        "bundleId"
    FROM app_vectors
    ORDER BY embedding<=>embed('a farming game')
    LIMIT 10

## Steps to achieve this
1. Use the [pghttp](https://github.com/pramsey/pgsql-http) extension to query the embedding endpoint.
2. Create an [SQL Function](https://www.postgresql.org/docs/current/sql-createfunction.html).
3. Verify that the above `embed` function yields the same result as embedding in python
4. Verify that the `ORDER BY embedding<=>embed('a farming game')` clause works without erros.

## Submission
You have 3 hours to complete the assignment.

Feel free to use any tool you think is helpful (Google, ChatGPT, CoPilot, etc).


Once done please notify your interviewer.


In [22]:
!pip install ipython-sql


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 2.3 MB/s eta 0:00:00


## embed function creation:

In [405]:
from sqlalchemy import create_engine, text

# Define the SQL command to create the function
sql_create_function = """
BEGIN;
CREATE OR REPLACE FUNCTION embed(text_input text)
RETURNS vector AS $$  -- Use the exact column type of 'embedding'
DECLARE
    api_response text;
    json_response json;
    embedding_array float8[];
    final_vector app_vectors.embedding%TYPE;  -- Declare variable of the embedding column type
BEGIN
    -- Make the HTTP GET request to the embedding service
    --SELECT content INTO api_response FROM http_get('http://emb:8080/embed?text=' || text_input);
    SELECT content INTO api_response FROM http_get('http://emb:8080/embed', jsonb_build_object('text', text_input));

    -- Parse the JSON response
    json_response := api_response::json;
    embedding_array := ARRAY(SELECT json_array_elements_text(json_response::json)::float8);

    final_vector := embedding_array;

    RETURN final_vector;
END;
$$ LANGUAGE plpgsql;
COMMIT;


"""

# Create engine
engine = create_engine('postgresql://postgres:argmax@pg:5432/postgres')

# Execute the command to create the function
print("Creating 'embed' function...")
with engine.connect() as connection:
    connection.execute(text(sql_create_function))
    print("Embedding function created successfully.")


Creating 'embed' function...
Embedding function created successfully.


## Results:

In [406]:
from sqlalchemy import create_engine
import pandas as pd

# Define your database connection URI
db_uri = 'postgresql://postgres:argmax@pg:5432/postgres'

# Create a SQLAlchemy engine
engine = create_engine(db_uri)

# Define your SQL query
sql_query = """
SELECT
    "bundleId"
FROM app_vectors
ORDER BY embedding <=> embed('a farming game')
LIMIT 10
"""

# Execute the SQL query and fetch the results into a pandas DataFrame
df = pd.read_sql(sql_query, engine)

# Print the resulting DataFrame
print(df)


              bundleId
0       se.ace.fishinc
1       se.ace.fishinc
2       se.ace.fishinc
3           1436213906
4           1436213906
5           1436213906
6  dovi.coalmining.inc
7  dovi.coalmining.inc
8  dovi.coalmining.inc
9           1582745578


# way to the goal
### no need to read this section, that was just my personal draft

In [51]:
import json
import requests

# Base URL for the embedding endpoint
EMBEDDING_ENDPOINT = "http://emb:8080/embed"


In [73]:
# Test the embed_text function
sample_text = "a farming game"
embedding = embed_text(sample_text)
print("Embedding for '{}' : {}".format(sample_text, embedding))

# Test the embed_func function
embedding_json = embed_func(sample_text)
print("Embedding JSON for '{}': {}".format(sample_text, embedding_json))


Embedding for 'a farming game' : [0.354161, 0.848121, -0.030284, -0.266633, -0.291369, -0.235802, -0.328547, 0.556077, 0.266265, 0.80911, 0.088049, 0.425911, 0.366686, 0.266709, -0.140682, -0.460998, -0.554848, -0.686548, -0.32312, 0.068696, 0.559805, 0.457103, -0.860526, -0.005353, -0.523179, 0.301315, -0.240179, -0.324041, 0.659639, 0.821582, -0.161332, 0.125691, -0.01421, -0.347641, -0.507953, -0.617259, 0.252902, -0.143336, -0.472049, -0.525402, 0.562244, -0.354457, 0.047799, -0.168605, 0.183767, -0.484756, 0.497892, -0.919021, 0.49797, -0.050206, 0.134188, -0.502712, -0.414337, -0.215174, -0.64883, -0.228308, -0.019863, 0.199011, -0.309924, 0.597036, 1.353649, -0.230922, 0.069312, -0.212216, 0.794984, 0.892149, -0.509315, -0.528928, -0.061134, -0.452766, -0.853116, 0.23235, -0.493771, -0.62472, 0.480888, 0.490589, 0.388387, -0.040939, -0.14579, 0.589986, 0.033822, 0.541735, 0.020558, -0.312139, -0.767122, 0.004014, 0.754478, 0.060997, -0.146129, -0.595552, -0.218311, 0.355091, -0.

In [378]:
# Define the function to get embeddings
sql_define_embed_function = text("""
BEGIN;
CREATE OR REPLACE FUNCTION embed(text_input text)
RETURNS float8[] AS $$
DECLARE
    api_response text;
    json_response json;
    embedding_vector float8[];
BEGIN
    -- Make the HTTP GET request to the embedding service
    SELECT content INTO api_response FROM http_get('http://emb:8080/embed?text=' || text_input);

    -- Parse the JSON response
    json_response := api_response::json;
    embedding_vector := ARRAY(SELECT json_array_elements_text(json_response::json));

    RETURN embedding_vector;
END;
$$ LANGUAGE plpgsql;
COMMIT;
""")

# Execute the command to define the function
with engine.connect() as connection:
    connection.execute(sql_define_embed_function)
    print("Embedding function created successfully.")

Embedding function created successfully.


In [403]:
from sqlalchemy import text

# Define the SQL command to drop the function
sql_drop_function = text("""
    DROP FUNCTION IF EXISTS embed(text);
""")

# Execute the command to drop the function
print("Dropping 'embed' function...")
with engine.connect() as connection:
    connection.execute(sql_drop_function)
    # Commit the transaction explicitly
    connection.commit()
    print("Embedding function dropped successfully.")

# Define the SQL query to check if the function exists
sql_check_function = text("""
    SELECT EXISTS (
        SELECT 1
        FROM pg_proc
        WHERE proname = 'embed'
    );
""")

# Execute the query to check if the function exists
print("Checking if 'embed' function exists...")
with engine.connect() as connection:
    result = connection.execute(sql_check_function)
    exists = result.scalar()

# Print the result
if exists:
    print("The 'embed' function still exists.")
else:
    print("The 'embed' function has been successfully dropped.")


Dropping 'embed' function...
Embedding function dropped successfully.
Checking if 'embed' function exists...
The 'embed' function has been successfully dropped.


In [126]:
# SQL command to define the function to insert data into the table
sql_define_insert_function = text("""
CREATE OR REPLACE FUNCTION insert_embedding_data(input_text text, embedding_vector float8[])
RETURNS void AS $$
BEGIN
    INSERT INTO embeddings (input_text, embedding_vector)
    VALUES (input_text, embedding_vector);
END;
$$ LANGUAGE plpgsql;
""")

# Execute the command to define the function
with engine.connect() as connection:
    connection.execute(sql_define_insert_function)
    print("Insert function created successfully.")

Insert function created successfully.


In [404]:
from sqlalchemy import text

# Define the SQL query to check if the function exists and its return type
sql_check_function = text("""
    SELECT proname, prorettype
    FROM pg_proc
    WHERE proname = 'embed';
""")

# Execute the query to check if the function exists and its return type
print("Checking if 'embed' function exists and its return type...")
with engine.connect() as connection:
    result = connection.execute(sql_check_function)
    row = result.fetchone()

# Print the result
if row:
    function_name, return_type = row
    print(f"The 'embed' function exists and returns type {return_type}.")
else:
    print("The 'embed' function does not exist.")



Checking if 'embed' function exists and its return type...
The 'embed' function does not exist.


In [172]:
import pandas as pd
from sqlalchemy import create_engine, text

# Create a SQLAlchemy engine
engine = engine = create_engine('postgresql://postgres:argmax@pg:5432/postgres')

# Define the SQL query to make the GET request
sql_query = """
SELECT content
FROM http_get('http://httpbun.com/ip');
"""

# Execute the SQL query and fetch the results into a pandas DataFrame
df = pd.read_sql(sql_query, engine)

# Print the resulting DataFrame
print(df)


                                content
0  {\n  "origin": "147.235.192.76"\n}\n


In [402]:
from sqlalchemy import create_engine
import pandas as pd

# Define your database connection URI
db_uri = 'postgresql://postgres:argmax@pg:5432/postgres'

# Create a SQLAlchemy engine
engine = create_engine(db_uri)

# Define your SQL query
sql_query = """
SELECT
    "bundleId"
FROM app_vectors
ORDER BY embedding <=> embed('a farming game')
LIMIT 10
"""

# Execute the SQL query and fetch the results into a pandas DataFrame
df = pd.read_sql(sql_query, engine)

# Print the resulting DataFrame
print(df)


              bundleId
0       se.ace.fishinc
1       se.ace.fishinc
2       se.ace.fishinc
3           1436213906
4           1436213906
5           1436213906
6  dovi.coalmining.inc
7  dovi.coalmining.inc
8  dovi.coalmining.inc
9           1582745578


In [319]:
from sqlalchemy import create_engine, text

# Define your database connection URI
db_uri = 'postgresql://postgres:argmax@pg:5432/postgres'

# Create a SQLAlchemy engine
engine = create_engine(db_uri)

# Define the SQL query to get the data type of the embedding column
sql_query = """
SELECT data_type
FROM information_schema.columns
WHERE table_name = 'app_vectors' AND column_name = 'embedding';
"""

# Compile the SQL query string into a SQLAlchemy text object
query = text(sql_query)

# Execute the SQL query and fetch the result
with engine.connect() as connection:
    result = connection.execute(query)
    data_type = result.fetchone()[0]

print("Data type of the 'embedding' column:", data_type)


Data type of the 'embedding' column: USER-DEFINED


In [366]:
from sqlalchemy import create_engine, text
from sqlalchemy.engine import Row

engine = create_engine('postgresql://postgres:argmax@pg:5432/postgres')

# Define the SQL query you want to execute
sql_query = """
SELECT typname, typtype, typcategory
FROM pg_type
WHERE typname = 'vector';  -- Replace 'vector' with the actual type name if known
"""

# Compile the SQL query string into a SQLAlchemy text object
query = text(sql_query)

# Execute the SQL query and fetch the result
with engine.connect() as connection:
    result = connection.execute(query)
    type_details = result.fetchone()  # Fetches the first row of the result
    if type_details:
        print("Type Name:", type_details.typname)  # Accessing by attribute
        print("Type Type:", type_details.typtype)
        print("Type Category:", type_details.typcategory)
    else:
        print("No details found for the specified type.")


Type Name: vector
Type Type: b
Type Category: U


# new aproach

In [215]:
import platform
import psycopg2

# Check the Python environment
python_version = platform.python_version()
print("Python Version:", python_version)

# Check the PostgreSQL server connection details
try:
    conn = psycopg2.connect(
        dbname="postgres",
        host="pg",
        port="5432"
    )
    cursor = conn.cursor()
    cursor.execute("SELECT version();")
    db_version = cursor.fetchone()[0]
    print("PostgreSQL Server Version:", db_version)
    cursor.close()
    conn.close()
except psycopg2.Error as e:
    print("Error connecting to PostgreSQL:", e)


Python Version: 3.11.6
Error connecting to PostgreSQL: connection to server at "pg" (172.19.0.2), port 5432 failed: fe_sendauth: no password supplied



In [229]:
import pandas as pd
from sqlalchemy import create_engine

# Create a SQLAlchemy engine
engine = create_engine('postgresql://postgres:argmax@pg:5432/postgres')

# Define the SQL query to make the GET request using http_get function
sql_query = """
SELECT content
FROM http_get('http://emb:8080/embed', '{"text": "A farming game"}'::jsonb);
"""

# Execute the SQL query and fetch the results into a pandas DataFrame
df = pd.read_sql(sql_query, engine)

# Print the resulting DataFrame
print(df)


                                             content
0  [0.354161,0.848121,-0.030284,-0.266633,-0.2913...


In [248]:
import pandas as pd
from sqlalchemy import create_engine

# Create a SQLAlchemy engine
engine = create_engine('postgresql://postgres:argmax@pg:5432/postgres')

# Define the SQL query
sql_query = """
SELECT content
FROM http_get('http://emb:8080/embed', '{"text": "A farming game"}'::jsonb);
"""

# Execute the SQL query and fetch the results into a pandas DataFrame
df = pd.read_sql(sql_query, engine)

# Extracting the vector from the 'content' column
vector_string = df['content'].iloc[0]  # Assuming there is only one row
vector_list = eval(vector_string)  # Convert the string representation to a list

# Extracting the vector from the list
embedding_vector = vector_list[1:]  # Skipping the ID

print("Embedding vector:", embedding_vector)



Embedding vector: [0.848121, -0.030284, -0.266633, -0.291369, -0.235802, -0.328547, 0.556077, 0.266265, 0.80911, 0.088049, 0.425911, 0.366686, 0.266709, -0.140682, -0.460998, -0.554848, -0.686548, -0.32312, 0.068696, 0.559805, 0.457103, -0.860526, -0.005353, -0.523179, 0.301315, -0.240179, -0.324041, 0.659639, 0.821582, -0.161332, 0.125691, -0.01421, -0.347641, -0.507953, -0.617259, 0.252902, -0.143336, -0.472049, -0.525402, 0.562244, -0.354457, 0.047799, -0.168605, 0.183767, -0.484756, 0.497892, -0.919021, 0.49797, -0.050206, 0.134188, -0.502712, -0.414337, -0.215174, -0.64883, -0.228308, -0.019863, 0.199011, -0.309924, 0.597036, 1.353649, -0.230922, 0.069312, -0.212216, 0.794984, 0.892149, -0.509315, -0.528928, -0.061134, -0.452766, -0.853116, 0.23235, -0.493771, -0.62472, 0.480888, 0.490589, 0.388387, -0.040939, -0.14579, 0.589986, 0.033822, 0.541735, 0.020558, -0.312139, -0.767122, 0.004014, 0.754478, 0.060997, -0.146129, -0.595552, -0.218311, 0.355091, -0.330841, 0.541868, 0.40548

In [234]:
from sqlalchemy import create_engine, text

# Create a SQLAlchemy engine
engine = create_engine('postgresql://postgres:argmax@pg:5432/postgres')

# Define the SQL query to check the data type accepted by <=>
sql_query = text("""
SELECT pg_typeof(embedding) AS embedding_data_type
FROM app_vectors
LIMIT 1;
""")

# Execute the SQL query and fetch the result
with engine.connect() as connection:
    result = connection.execute(sql_query)
    data_type = result.scalar()

# Print the data type
print("Data type accepted by <=> operator:", data_type)


Data type accepted by <=> operator: vector


In [399]:
from sqlalchemy import create_engine, text

# Define the SQL command to create the function
sql_create_function = """
BEGIN;
CREATE OR REPLACE FUNCTION embed(text_input text)
RETURNS vector AS $$  -- Use the exact column type of 'embedding'
DECLARE
    api_response text;
    json_response json;
    embedding_array float8[];
    final_vector app_vectors.embedding%TYPE;  -- Declare variable of the embedding column type
BEGIN
    -- Make the HTTP GET request to the embedding service
    --SELECT content INTO api_response FROM http_get('http://emb:8080/embed?text=' || text_input);
    SELECT content INTO api_response FROM http_get('http://emb:8080/embed', jsonb_build_object('text', text_input));

    -- Parse the JSON response
    json_response := api_response::json;
    embedding_array := ARRAY(SELECT json_array_elements_text(json_response::json)::float8);

    final_vector := embedding_array;

    RETURN final_vector;
END;
$$ LANGUAGE plpgsql;
COMMIT;


"""

# Create engine
engine = create_engine('postgresql://postgres:argmax@pg:5432/postgres')

# Execute the command to create the function
print("Creating 'embed' function...")
with engine.connect() as connection:
    connection.execute(text(sql_create_function))
    print("Embedding function created successfully.")


Creating 'embed' function...
Embedding function created successfully.


In [329]:
import pandas as pd
from sqlalchemy import create_engine

# Define your database connection URI
db_uri = 'postgresql://postgres:argmax@pg:5432/postgres'

# Create a SQLAlchemy engine
engine = create_engine(db_uri)

# Define your SQL query to retrieve the first 10 rows of the app_vectors table
sql_query = """
SELECT *
FROM app_vectors
LIMIT 10
"""

# Execute the SQL query and fetch the results into a pandas DataFrame
df = pd.read_sql(sql_query, engine)

# Print the resulting DataFrame
print(df)


   id                     bundleId  \
0   1             com.loop.match3d   
1   2       com.YayySAL.DodgeAgent   
2   3        com.tintash.nailsalon   
3   4        com.tilegarden.match3   
4   6   com.AppIdeas.LevelUpRunner   
5  17                   1582745578   
6  18                   1569586264   
7   5  com.kamilbilge.ropesavior3d   
8   7               se.ace.fishinc   
9   8      com.volt.dresstoimpress   

                                             content  \
0  Get ready for a new, challenging and original ...   
1  Your mission, should you choose to accept it, ...   
2  It is manicure madness over here and it�s your...   
3  \nIf you enjoy playing Match 3 tile & mahjong ...   
4  Are you a real hero? Are you the strongest, th...   
5  Dominate the coal industry and become the next...   
6  Choose the appropriate outfit to make it throu...   
7  Become the hero we need in this tangled puzzle...   
8  Gather your riches and travel across the open ...   
9  \nChoose the appro